# Learning Rate Scheduling
:label:`sec_scheduler`

So far we primarily focused on optimization *algorithms* for how to update the weight vectors rather than on the *rate* at which they are being updated. Nonetheless, adjusting the learning rate is often just as important as the actual algorithm. There are a number of aspects to consider:

* Most obviously the *magnitude* of the learning rate matters. If it is too large, optimization diverges, if it is too small, it takes too long to train or we end up with a suboptimal result. We saw previously that the condition number of the problem matters (see e.g., :numref:`sec_momentum` for details). Intuitively it is the ratio of the amount of change in the least sensitive direction vs. the most sensitive one.
* Secondly, the rate of decay is just as important. If the learning rate remains large we may simply end up bouncing around the minimum and thus not reach optimality. :numref:`sec_minibatch_sgd` discussed this in some detail and we analyzed performance guarantees in :numref:`sec_sgd`. In short, we want the rate to decay, but probably more slowly than $\mathcal{O}(t^{-\frac{1}{2}})$ which would be a good choice for convex problems.
* Another aspect that is equally important is *initialization*. This pertains both to how the parameters are set initially (review :numref:`sec_numerical_stability` for details) and also how they evolve initially. This goes under the moniker of *warmup*, i.e., how rapidly we start moving towards the solution initially. Large steps in the beginning might not be beneficial, in particular since the initial set of parameters is random. The initial update directions might be quite meaningless, too.
* Lastly, there are a number of optimization variants that perform cyclical learning rate adjustment. This is beyond the scope of the current chapter. We recommend the reader to review details in :cite:`Izmailov.Podoprikhin.Garipov.ea.2018`, e.g., how to obtain better solutions by averaging over an entire *path* of parameters.

Given the fact that there is a lot of detail needed to manage learning rates, most deep learning frameworks have tools to deal with this automatically. In the current chapter we will review the effects that different schedules have on accuracy and also show how this can be managed efficiently via a *learning rate scheduler*. 

In DJL we will be referring to these as learning rate trackers.

## Toy Problem

We begin with a toy problem that is cheap enough to compute easily, yet sufficiently nontrivial to illustrate some of the key aspects. For that we pick a slightly modernized version of LeNet (`relu` instead of `sigmoid` activation, MaxPooling rather than AveragePooling), as applied to Fashion-MNIST. Moreover, we hybridize the network for performance. Since most of the code is standard we just introduce the basics without further detailed discussion. See :numref:`chap_cnn` for a refresher as needed.


In [1]:
%use @file[../djl.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.util.GradDescUtils.plotGammas
import jp.live.ugai.d2j.util.GradDescUtils.train2d
import jp.live.ugai.d2j.util.GradDescUtils.showTrace2d
import jp.live.ugai.d2j.util.TrainingChapter11.getDataCh11
import jp.live.ugai.d2j.util.TrainingChapter11.plotLossEpoch
import jp.live.ugai.d2j.util.TrainingChapter11.trainCh11
import jp.live.ugai.d2j.util.TrainingChapter11.trainConciseCh11
import jp.live.ugai.d2j.util.LossTime

In [2]:
import ai.djl.basicdataset.cv.classification.*;
import ai.djl.metric.Metrics

In [3]:
val net = SequentialBlock();

net.add(Conv2d.builder()
        .setKernelShape(Shape(5, 5))
        .optPadding(Shape(2, 2))
        .setFilters(1)
        .build());
net.add(Activation.reluBlock());
net.add(Pool.maxPool2dBlock(Shape(2, 2), Shape(2, 2)));
net.add(Conv2d.builder()
        .setKernelShape(Shape(5, 5))
        .setFilters(1)
        .build());
net.add(Blocks.batchFlattenBlock());
net.add(Activation.reluBlock());
net.add(Linear.builder().setUnits(120).build());
net.add(Activation.reluBlock());
net.add(Linear.builder().setUnits(84).build());
net.add(Activation.reluBlock());
net.add(Linear.builder().setUnits(10).build());

SequentialBlock {
	Conv2d
	ReLU
	maxPool2d
	Conv2d
	batchFlatten
	ReLU
	Linear
	ReLU
	Linear
	ReLU
	Linear
}

In [4]:
val batchSize = 256;
val trainDataset = FashionMnist.builder()
        .optUsage(Dataset.Usage.TRAIN)
        .setSampling(batchSize, false)
        .optLimit(Long.MAX_VALUE)
        .build();

val testDataset = FashionMnist.builder()
        .optUsage(Dataset.Usage.TEST)
        .setSampling(batchSize, false)
        .optLimit(Long.MAX_VALUE)
        .build();

In [5]:
var trainLoss = listOf<Double>()
var testAccuracy = listOf<Double>()
var epochCount= listOf<Int>()
var trainAccuracy= listOf<Double>()

fun train(trainIter:RandomAccessDataset , testIter:RandomAccessDataset ,
                             numEpochs: Int, trainer:Trainer) {
    epochCount = List<Int>(numEpochs) { it+1 }
    var avgTrainTimePerEpoch = 0.0
    val evaluatorMetrics = mutableMapOf<String, List<Double>>()

    trainer.setMetrics(Metrics());

    EasyTrain.fit(trainer, numEpochs, trainIter, testIter);

    var metrics = trainer.getMetrics();

    trainer.getEvaluators().forEach{ evaluator -> 
                evaluatorMetrics.put("train_epoch_" + evaluator.getName(), metrics.getMetric("train_epoch_" + evaluator.getName()).map { it.value });
                evaluatorMetrics.put("validate_epoch_" + evaluator.getName(), metrics.getMetric("validate_epoch_" + evaluator.getName()).map { it.value });
            }

    avgTrainTimePerEpoch = metrics.mean("epoch");

    trainLoss = evaluatorMetrics.get("train_epoch_SoftmaxCrossEntropyLoss")!!
    trainAccuracy = evaluatorMetrics!!.get("train_epoch_Accuracy")!!
    testAccuracy = evaluatorMetrics.get("validate_epoch_Accuracy")!!

    print("loss %.3f,".format(trainLoss[numEpochs-1]))
    print(" train acc %.3f,".format(trainAccuracy[numEpochs-1]))
    println(" test acc %.3f".format(testAccuracy[numEpochs-1]))
    println("%.1f examples/sec ".format(trainIter.size() / (avgTrainTimePerEpoch / Math.pow(10.0, 9.0))))
}

Let us have a look at what happens if we invoke this algorithm with default settings, such as a learning rate of $0.3$ and train for $30$ iterations. Note how the training accuracy keeps on increasing while progress in terms of test accuracy stalls beyond a point. The gap between both curves indicates overfitting.


In [6]:
val lr = 0.3f;
val numEpochs = Integer.getInteger("MAX_EPOCH", 10);

val model = Model.newInstance("Modern LeNet");
model.setBlock(net);

val loss = Loss.softmaxCrossEntropyLoss();
val lrt = Tracker.fixed(lr);
val sgd = Optimizer.sgd().setLearningRateTracker(lrt).build();

val config = DefaultTrainingConfig(loss)
        .optOptimizer(sgd) // Optimizer
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()); // Logging

val trainer = model.newTrainer(config);
trainer.initialize(Shape(1, 1, 28, 28));

train(trainDataset, testDataset, numEpochs, trainer);

Training:    100% |████████████████████████████████████████| Accuracy: 0.62, SoftmaxCrossEntropyLoss: 1.03
Validating:  100% |████████████████████████████████████████|�███████                        |�████████              |ing:   89% |████████████████████████████████████    |
Training:    100% |████████████████████████████████████████| Accuracy: 0.79, SoftmaxCrossEntropyLoss: 0.56
Validating:  100% |████████████████████████████████████████|��█████████████████████████████████     |
Training:    100% |████████████████████████████████████████| Accuracy: 0.82, SoftmaxCrossEntropyLoss: 0.49
Validating:  100% |████████████████████████████████████████|█                   |��██████████████        |
Training:    100% |████████████████████████████████████████| Accuracy: 0.84, SoftmaxCrossEntropyLoss: 0.45
Validating:  100% |████████████████████████████████████████|█                   |█████████████████████       |
Training:    100% |████████████████████████████████████████| Accuracy: 0.85, Soft

In [7]:
val data = mapOf("x" to epochCount, "y0" to trainLoss, "y1" to trainAccuracy, "y2" to testAccuracy)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot += geomLine(color="blue") { x = "x" ; y="y1"}
plot += geomLine(color="green") { x= "x" ; y="y2"}
plot + ggsize(500,400)

## Trackers

One way of adjusting the learning rate is to set it explicitly at each step. We could adjust it downward after every epoch (or even after every minibatch), e.g., in a dynamic manner in response to how optimization is progressing. 

We, however, can't directly change the learning rate with the trainer after it has already been created. What we can do instead is create a tracker to do this for us.

When invoked with the number of updates it returns the appropriate value of the learning rate. Let us define a simple one that sets the learning rate to $\eta = \eta_0 (t + 1)^{-\frac{1}{2}}$.

In [8]:
class SquareRootTracker {
    var lr : Float
    constructor() {
        lr = 0.1f
    }
    
    constructor(learningRate: Float) {
        lr = learningRate;        
    }
    fun getNewLearningRate(numUpdate: Int): Float {
        return lr * Math.pow(numUpdate.toDouble() + 1, -0.5).toFloat()
    }
}

Note: This is not a drop in replacement for a standard Learning Rate Tracker (LRT). 
This is just a simple example to give a better understanding of how they work.

Let us plot its behavior over a range of values.


In [9]:
val tracker = SquareRootTracker()

val epochs = List<Int>(numEpochs) { it }
val learningRates = mutableListOf<Float>()
for (i in 0 until numEpochs) {
    learningRates.add(tracker.getNewLearningRate(i))
}

val data = mapOf("x" to epochs, "y0" to  learningRates)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot + ggsize(500,400)

Now let us see how this plays out for training on Fashion-MNIST. We can't actually do it directly, but we can see how the curve would look theoretically.

This looks like it works quite a bit better than before. Two things stand out: the curve was rather more smooth than previously. Secondly, there was less overfitting. Unfortunately it is not a well-resolved question as to why certain strategies lead to less overfitting in *theory*. There is some argument that a smaller stepsize will lead to parameters that are closer to zero and thus simpler. However, this does not explain the phenomenon entirely since we do not really stop early but simply reduce the learning rate gently.

## Policies

While we cannot possibly cover the entire variety of learning rate trackers, we attempt to give a brief overview of popular policies below. Common choices are polynomial decay and piecewise constant schedules. Beyond that, cosine learning rate schedules have been found to work well empirically on some problems. Lastly, on some problems it is beneficial to warm up the optimizer prior to using large learning rates.

### Factor Tracker

One alternative to a polynomial decay would be a multiplicative one, that is $\eta_{t+1} \leftarrow \eta_t \cdot \alpha$ for $\alpha \in (0, 1)$. To prevent the learning rate from decaying beyond a reasonable lower bound the update equation is often modified to $\eta_{t+1} \leftarrow \mathop{\mathrm{max}}(\eta_{\mathrm{min}}, \eta_t \cdot \alpha)$.


In [10]:
class DemoFactorTracker {
    val baseLr: Float
    val stopFactorLr: Float
    val factor: Float
    constructor(factor: Float, stopFactorLr: Float, baseLr: Float) {
        this.factor = factor;
        this.stopFactorLr = stopFactorLr;
        this.baseLr = baseLr;
    }
    constructor() {
        factor = 1f
        stopFactorLr =1e-7f
        baseLr = 0.1f
    }
    fun getNewLearningRate(numUpdate: Int): Float {
        return lr * Math.pow(numUpdate.toDouble() + 1, -0.5).toFloat()
    }
}

In [11]:
val tracker = DemoFactorTracker(0.9f, 1e-2f, 2f)

val numEpochs = 50
val epochs = List<Int>(numEpochs) { it }
val learningRates = mutableListOf<Float>()
for (i in 0 until numEpochs) {
    learningRates.add(tracker.getNewLearningRate(i))
}

val data = mapOf("x" to epochs, "y0" to  learningRates)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot + ggsize(500,400)

This can also be accomplished by a built-in scheduler in DJL via the `LearningRateTracker.factorTracker()` builder. It takes a few more parameters, such as warmup period, warmup mode (linear or constant), the maximum number of desired updates, etc.; Going forward we will use the built-in schedulers as appropriate and only explain their functionality here. 

### Multi Factor Scheduler

A common strategy for training deep networks is to keep the learning rate piecewise constant and to decrease it by a given amount every so often. That is, given a set of times when to decrease the rate, such as $s = \{5, 10, 20\}$ decrease $\eta_{t+1} \leftarrow \eta_t \cdot \alpha$ whenever $t \in s$. Assuming that the values are halved at each step we can implement this as follows.


In [12]:
val tracker = Tracker.multiFactor()
        .setSteps(intArrayOf(5, 30))
        .optFactor(0.5f)
        .setBaseValue(0.5f)
        .build()

val numEpochs = 50;
val epochs = List<Int>(numEpochs) { it }
val learningRates = mutableListOf<Float>()
for (i in 0 until numEpochs) {
    learningRates.add(tracker.getNewValue(i))
}

val data = mapOf("x" to epochs, "y0" to  learningRates)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot + ggsize(500,400)

The intuition behind this piecewise constant learning rate schedule is that one lets optimization proceed until a stationary point has been reached in terms of the distribution of weight vectors. Then (and only then) do we decrease the rate such as to obtain a higher quality proxy to a good local minimum. The example below shows how this can produce ever slightly better solutions.


In [13]:
val numEpochs = 10

val model = Model.newInstance("Modern LeNet");
model.setBlock(net);

val loss = Loss.softmaxCrossEntropyLoss();
val sgd = Optimizer.sgd().setLearningRateTracker(tracker).build();

val config = DefaultTrainingConfig(loss)
        .optOptimizer(sgd) // Optimizer
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()); // Logging

val trainer = model.newTrainer(config);
trainer.initialize(Shape(1, 1, 28, 28));

train(trainDataset, testDataset, numEpochs, trainer);
val data = mapOf("x" to epochCount, "y0" to trainLoss, "y1" to trainAccuracy, "y2" to testAccuracy)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot += geomLine(color="blue") { x = "x" ; y="y1"}
plot += geomLine(color="green") { x= "x" ; y="y2"}
plot + ggsize(500,400)

Training:    100% |████████████████████████████████████████| Accuracy: 0.88, SoftmaxCrossEntropyLoss: 0.32
Validating:  100% |████████████████████████████████████████|��████████                  |�████████         |██████████████████ |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.31
Validating:  100% |████████████████████████████████████████|��████████                  |�████████         |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.30
Validating:  100% |████████████████████████████████████████|   |�██████████████████████            |�████████████  |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.29
Validating:  100% |████████████████████████████████████████|�███████                        |�███████████████             |�████████████  |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, 

In [14]:
val data = mapOf("x" to epochCount, "y0" to trainLoss, "y1" to trainAccuracy, "y2" to testAccuracy)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot += geomLine(color="blue") { x = "x" ; y="y1"}
plot += geomLine(color="green") { x= "x" ; y="y2"}
plot + ggsize(500,400)

### Cosine Tracker

A rather perplexing heuristic was proposed by :cite:`Loshchilov.Hutter.2016`. It relies on the observation that we might not want to decrease the learning rate too drastically in the beginning and moreover, that we might want to "refine" the solution in the end using a very small learning rate. This results in a cosine-like tracker with the following functional form for learning rates in the range $t \in [0, T]$.

$$\eta_t = \eta_T + \frac{\eta_0 - \eta_T}{2} \left(1 + \cos(\pi t/T)\right)$$

Here $\eta_0$ is the initial learning rate, $\eta_T$ is the target rate at time $T$. Furthermore, for $t > T$ we simply pin the value to $\eta_T$ without increasing it again. In the following example, we set the max update step $T = 20$.


In [15]:
class DemoCosineTracker {
    var baseLr : Float
    var finalLr : Float
    var maxUpdate : Int
    constructor() {
        baseLr = 0.5f
        finalLr =0.01f
        maxUpdate =20
    }
    constructor(baseLr: Float, finalLr: Float, maxUpdate: Int) {
        this.baseLr = baseLr;
        this.finalLr = finalLr;
        this.maxUpdate = maxUpdate;
    }
    fun getNewLearningRate(numUpdate: Int): Float {
        if (numUpdate > maxUpdate) {
            return finalLr;
        }
        // Scale the curve to smoothly transition
        val step = (baseLr - finalLr) / 2 * (1 + Math.cos(Math.PI * numUpdate / maxUpdate));
        return finalLr + step.toFloat()
    }
}

In [16]:
val tracker = DemoCosineTracker(0.5f, 0.01f, 20);

val numEpochs = 30
val epochs = List<Int>(numEpochs) { it }
val learningRates = List<Float>(numEpochs) { tracker.getNewLearningRate(it) }
val data = mapOf("x" to epochs, "y0" to  learningRates)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot + ggsize(500,400)

In the context of computer vision this schedule *can* lead to improved results. Note, though, that such improvements are not guaranteed (as can be seen below).


In [17]:
val cosineTracker = Tracker.cosine()
                            .setBaseValue(0.5f)
                            .optFinalValue(0.01f)
                            .setMaxUpdates(20)
                            .build();

val loss = Loss.softmaxCrossEntropyLoss();
val sgd = Optimizer.sgd().setLearningRateTracker(cosineTracker).build();

val config = DefaultTrainingConfig(loss)
        .optOptimizer(sgd) // Optimizer
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()); // Logging

val trainer = model.newTrainer(config);
trainer.initialize(Shape(1, 1, 28, 28));

val numEpochs = 10
val epochs = List<Int>(numEpochs) { it }
train(trainDataset, testDataset, numEpochs, trainer)
val data = mapOf("x" to epochs, "y0" to trainLoss, "y1" to trainAccuracy, "y2" to testAccuracy)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot += geomLine(color="blue") { x = "x" ; y="y1"}
plot += geomLine(color="green") { x= "x" ; y="y2"}
plot + ggsize(500,400)

Training:    100% |████████████████████████████████████████| Accuracy: 0.85, SoftmaxCrossEntropyLoss: 0.43
Validating:  100% |████████████████████████████████████████|   |��█████████████████████████████████     |
Training:    100% |████████████████████████████████████████| Accuracy: 0.88, SoftmaxCrossEntropyLoss: 0.34
Validating:  100% |████████████████████████████████████████|��████████                  |�████████         |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.31
Validating:  100% |████████████████████████████████████████|�████████              |█    |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.29
Validating:  100% |████████████████████████████████████████|�███████                        |�███████████████             |�████████████  |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.28
Validating:  100% |

### Warmup

In some cases initializing the parameters is not sufficient to guarantee a good solution. This particularly a problem for some advanced network designs that may lead to unstable optimization problems. We could address this by choosing a sufficiently small learning rate to prevent divergence in the beginning. Unfortunately this means that progress is slow. Conversely, a large learning rate initially leads to divergence.

A rather simple fix for this dilemma is to use a warmup period during which the learning rate *increases* to its initial maximum and to cool down the rate until the end of the optimization process. For simplicity one typically uses a linear increase for this purpose. This leads to a schedule of the form indicated below.


In [18]:
class CosineWarmupTracker {
    var baseLr: Float
    var finalLr: Float
    var maxUpdate: Int
    var warmUpSteps : Int
    var warmUpBeginValue : Float = 0.0f
    var warmUpFinalValue : Float = 0.0f
    
    constructor() {
        baseLr = 0.5f
        finalLr = 0.01f
        maxUpdate = 20
        warmUpSteps =5
    }
    
    constructor (baseLr: Float, finalLr: Float, maxUpdate: Int, warmUpSteps: Int) {
        this.baseLr = baseLr;
        this.finalLr = finalLr;
        this.maxUpdate = maxUpdate;
        this.warmUpSteps = 5;
        this.warmUpBeginValue = 0f;
    }
    
    fun getNewLearningRate(numUpdate: Int): Float {
        if (numUpdate <= warmUpSteps) {
            return getWarmUpValue(numUpdate);
        }
        if (numUpdate > maxUpdate) {
            return finalLr;
        }
        // Scale the cosine curve to fit smoothly with the warmup steps
        var step = (baseLr - finalLr) / 2 * (1 + 
            Math.cos(Math.PI * (numUpdate - warmUpSteps) / (maxUpdate - warmUpSteps)));
        return finalLr + step.toFloat();
    }
    
    fun getWarmUpValue(numUpdate: Int): Float {
        // Linear warmup
        return warmUpBeginValue + (baseLr - warmUpBeginValue) * numUpdate / warmUpSteps;
    }
}

In [19]:
val tracker = CosineWarmupTracker(0.5f, 0.01f, 20, 5);

val epochs = List<Int>(numEpochs) { it }
val learningRates = List<Float>(numEpochs) { tracker.getNewLearningRate(it) }
val data = mapOf("x" to epochs, "y0" to  learningRates)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot + ggsize(500,400)

Note that the network converges better initially (in particular observe the performance during the first 5 epochs).

Additionally, we still use a total of 20 max updates, but the 1st
5 are dedicated to the warmup steps. The cosine curve will then be
squeezed into the 15 steps relative to the earlier 20 steps.

In [20]:
val cosineTracker = Tracker.cosine()
        .setBaseValue(0.5f)
        .optFinalValue(0.01f)
        .setMaxUpdates(15)
        .build();

val warmupCosine = Tracker.warmUp()
        .optWarmUpSteps(5)
        .setMainTracker(cosineTracker)
        .build();

val loss = Loss.softmaxCrossEntropyLoss();
val sgd = Optimizer.sgd().setLearningRateTracker(warmupCosine).build();

val config = DefaultTrainingConfig(loss)
        .optOptimizer(sgd) // Optimizer
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()); // Logging

val trainer = model.newTrainer(config);
trainer.initialize(Shape(1, 1, 28, 28));

train(trainDataset, testDataset, numEpochs, trainer);
val epochs = List<Int>(numEpochs) { it }
val data = mapOf("x" to epochs, "y0" to trainLoss, "y1" to trainAccuracy, "y2" to testAccuracy)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot += geomLine(color="blue") { x = "x" ; y="y1"}
plot += geomLine(color="green") { x= "x" ; y="y2"}
plot + ggsize(500,400)

Training:    100% |████████████████████████████████████████| Accuracy: 0.80, SoftmaxCrossEntropyLoss: 0.56
Validating:  100% |████████████████████████████████████████|�█               |��█████████████████████████████████     |
Training:    100% |████████████████████████████████████████| Accuracy: 0.86, SoftmaxCrossEntropyLoss: 0.39
Validating:  100% |████████████████████████████████████████|ting:   35% |███████████████                         |�█               |��█████████████████████████████████     |
Training:    100% |████████████████████████████████████████| Accuracy: 0.87, SoftmaxCrossEntropyLoss: 0.34
Validating:  100% |████████████████████████████████████████|�███████                        |�████████              |��██████   |
Training:    100% |████████████████████████████████████████| Accuracy: 0.88, SoftmaxCrossEntropyLoss: 0.32
Validating:  100% |████████████████████████████████████████||██          |
Training:    100% |████████████████████████████████████████| Accuracy: 0.

In [21]:
val sgd = Optimizer.sgd().setLearningRateTracker(cosineTracker).build();

val config = DefaultTrainingConfig(loss)
        .optOptimizer(sgd) // Optimizer
        .addEvaluator(Accuracy()) // Model Accuracy
        .addTrainingListeners(*TrainingListener.Defaults.logging()); // Logging

val trainer = model.newTrainer(config);
trainer.initialize(Shape(1, 1, 28, 28));

train(trainDataset, testDataset, numEpochs, trainer);
val epochs = List<Int>(numEpochs) { it }
val data = mapOf("x" to epochs, "y0" to trainLoss, "y1" to trainAccuracy, "y2" to testAccuracy)
var plot = letsPlot(data)
plot += geomLine(color="red") { x = "x"; y="y0"}
plot += geomLine(color="blue") { x = "x" ; y="y1"}
plot += geomLine(color="green") { x= "x" ; y="y2"}
plot + ggsize(500,400)

Training:    100% |████████████████████████████████████████| Accuracy: 0.88, SoftmaxCrossEntropyLoss: 0.33
Validating:  100% |████████████████████████████████████████|�███████                        |�████████              |��██████   |
Training:    100% |████████████████████████████████████████| Accuracy: 0.89, SoftmaxCrossEntropyLoss: 0.29
Validating:  100% |████████████████████████████████████████|�███████                        |�███████████████             |��██████   |
Training:    100% |████████████████████████████████████████| Accuracy: 0.90, SoftmaxCrossEntropyLoss: 0.28
Validating:  100% |███████████�████████████████████████████|��████████                  |█████████████████████       |�████████████████████████████|
Training:    100% |████████████████████████████████████████| Accuracy: 0.90, SoftmaxCrossEntropyLoss: 0.27
Validating:  100% |████████████████████████████████████████|�████████████████                 |█████████████████████       |
Training:    100% |█████████████

Warmup can be applied to any scheduler (not just cosine). For a more detailed discussion of learning rate schedules and many more experiments see also :cite:Gotmare.Keskar.Xiong.ea.2018. In particular they find that a warmup phase limits the amount of divergence of parameters in very deep networks. This makes intuitively sense since we would expect significant divergence due to random initialization in those parts of the network that take the most time to make progress in the beginning.

## Summary
* Decreasing the learning rate during training can lead to improved accuracy and (most perplexingly) reduced overfitting of the model.
* A piecewise decrease of the learning rate whenever progress has plateaued is effective in practice. Essentially this ensures that we converge efficiently to a suitable solution and only then reduce the inherent variance of the parameters by reducing the learning rate.
* Cosine schedulers are popular for some computer vision problems.\n",
* A warmup period before optimization can prevent divergence.\n",
* Optimization serves multiple purposes in deep learning. Besides minimizing the training objective, different choices of optimization algorithms and learning rate scheduling can lead to rather different amounts of generalization and overfitting on the test set (for the same amount of training error).\n",

## Exercises

1. Experiment with the optimization behavior for a given fixed learning rate. What is the best model you can obtain this way?
1. How does convergence change if you change the exponent of the decrease in the learning rate?
1. Apply the cosine scheduler to large computer vision problems, e.g., training ImageNet. How does it affect performance relative to other schedulers?
1. How long should warmup last?
1. Can you connect optimization and sampling? Start by using results from :cite:`Welling.Teh.2011` on Stochastic Gradient Langevin Dynamics.